## И еще про классы в Python

In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [9]:
df = pd.read_csv('data.csv')
df

,a,b,c,d,e,f
0,105,6300,6,1,3060,6300
1,225,2700,5,2,3780,79200
2,255,26100,2,2,2340,79200
3,105,6300,3,1,3060,6300
4,255,26100,2,2,2340,79200
5,105,6300,9,1,3060,6300
6,95,5700,9,2,2460,5700


In [10]:
df['b'] /= 60

In [11]:
target = ['f']

x = df[['a', 'b']]
y = df[target]

In [12]:
reg = LinearRegression()
reg.fit(x, y)

LinearRegression()

In [13]:
reg.coef_

array([[574.10469909, -43.99752679]])

In [14]:
df

,a,b,c,d,e,f
0,105,105.0,6,1,3060,6300
1,225,45.0,5,2,3780,79200
2,255,435.0,2,2,2340,79200
3,105,105.0,3,1,3060,6300
4,255,435.0,2,2,2340,79200
5,105,105.0,9,1,3060,6300
6,95,95.0,9,2,2460,5700


### Наследование

Вспомним игрушечный пример с прошлого семинара

In [1]:
class Parent:
    def __init__(self, letter='D'):
        self._letter = letter
        
    def get_letter(self):
        return self._letter
    
class Child(Parent):
    def __init__(self, parent_letter='D', child_letter='B'):
        super().__init__(parent_letter)
        print(f'parent inited {self._letter}')
        self.new_letter = child_letter
        
    def get_child_letter(self):
        return self.new_letter

#### Предположим, вы хотите отнаследоваться от стандартного типа. Что может пойти не так

In [17]:
class MyDict(dict):
    def __setitem__(self, k, v) -> None:
        super().__setitem__(k, [v] * 2)

In [16]:
a = MyDict(one=1)
a['two'] = 2
a.update({'third': 4})
a

{'one': 1, 'two': [2, 2], 'third': 4}

In [18]:
class TroubleDict(dict):
    def __getitem__(self, k):
        return 42

In [20]:
bd = TroubleDict(a = 'answer')
bd[a]

42

#### Как надо?

In [22]:
from collections import UserDict

In [24]:
class MyDict(UserDict):
    def __setitem__(self, k, v) -> None:
        super().__setitem__(k, [v] * 2)

In [25]:
dd = MyDict(one=1)
dd

{'one': [1, 1]}

### ABC

`Types are defined by their supported operations`

Для того, чтобы что-то работало как последовательность, не обязательно определять много методов. Достаточно объявить `__getitem__`

In [26]:
class Vovels:
    def __getitem__(self, i):
        return 'AIEUO'[i]

vovels = Vovels()
vovels[0], vovels[-1]

('A', 'O')

In [27]:
for v in vovels: print(v)

A
I
E
U
O


Однако несмотря на простоту, мы не определили len! Казалось бы, один из базовых методов, но можно и без него

In [29]:
len(vovels)

TypeError: object of type 'Vovels' has no len()

Можно сказать, что в данном случае мы реализовали часть *динамического протокола* последовательности, и при этом все равно смогли использовать ее функционал. Однако, в таком случае мы можем столкнуться с проблемами, когда передадим нашу "последовательность" куда-то, где от нее будут ожидать реализацию хотя бы метода длины

Это отличный пример **duck typing** -- игнорирование конкретного типа объекта, если он определяет конкретные необходимые методы

Нам не обязательно создавать наследника класса последовательности, чтобы использовать объект так, как мы хотим. Python делает много работы за нас в этом случае

Но что если мы явно хотим сделать некоторые методы обязательными? Здесь нам на помощь приходят **abstract base classes**
https://docs.python.org/3/library/abc.html

#### **Goose typing**:
- Наследуемся от ABC чтобы явно продемонстрировать следование объявленному интерфейсу
- Проверка типа isinstance и issubclass осуществляется для интерфейса

In [30]:
from collections import namedtuple, abc

class SequenceVovels(abc.Sequence):
    def __getitem__(self, i):
        return 'AIEUO'[i]

In [31]:
vov = SequenceVovels()

TypeError: Can't instantiate abstract class SequenceVovels with abstract methods __len__

In [32]:
class SequenceVovels(abc.Sequence):
    
    VOVELS = 'AIEUO'
    
    def __getitem__(self, i):
        return VOVELS[i]
    
    def __len__(self):
        return len(self.VOVELS)

In [33]:
vov = SequenceVovels()

#### Можно объявить свой ABC

In [34]:
import abc
class MyABC(abc.ABC):
    
    @abc.abstractmethod
    def do_smth(self, a):
        """does something"""
    
    @staticmethod
    @abc.abstractmethod  # should be inner, order matters
    def do_smth_static(a):
        """a staticmethod that does something"""
        
    @classmethod
    @abc.abstractmethod  # should be inner, order matters
    def do_smth_class(cls, a):
        """a classmethod that does something"""

#### Варианты проверки типов

Предположим, у нас есть некоторый объект `maybe_complex` и мы хотим проверить, можно ли использовать его как Complex number

**1 способ**

```python
from typing import SupportsComplex

if isinstance(maybe_comlex, (complex, SupportsComplex)):
    # do smth that requires our object to be complex
    else:
        raise TypeError('maybe_complex should be convertible to complex')
```

**2 способ**

```python

import numbers

if isinstance(maybe_comlex, (numbers.Complex)):
    # do smth that requires our object to be complex
    else:
        raise TypeError('maybe_complex should be an instance of complex')
```

**3 способ aka Fail Fast or EAFP** *‘Easier to Ask for Forgiveness than Permission’*



```python

try:
    c = complex(maybe_complex)
except TypeError as e:
    raise TypeError('maybe_complex should be convertible to complex')
```

Несколько советов про наследование:

1. Не переиспользуйте наследование for fun там, где этого можно избежать
2. Понимайте, почему в конкретном месте используется наследование
3. Если вам важно соблюдение интерфейса, обозначьте это явно с помощью ABC
4. Будьте осторожны с наследованием от стандартных типов. Используйте модуль Collections

## Регулярные выражения

Полезные ссылки:

https://habr.com/ru/post/349860/

https://uneex.org/HSE/ProgrammingOS/15_Regexp
    
https://regexone.com/    
    
https://ravesli.com/regulyarnye-vyrazheniya-osnovy/
    
https://regex101.com/

## Fun regexp tasks

Пример с логом apache

`cat error.log.14 | grep -P '[A-Z0-9]+: .*restart'`

### E-mails

Вы забеспокоились о приватности данных ваших однокурсников и преподавателей и решили тайком удалить все их личные почты из общего файла. При этом, вы хотите, чтобы с ними можно было связаться, поэтому университетские почты вы в файле оставляете.

Вывести все упомянутые почты. Все почты, не отсносящиеся к @hse.ru или @edu.hse.ru, заменить на фразу "Pr1v@cY REstorED"

PS Попробуйте сделать это в командной строчке линукса. Вам поможет команда sed

In [36]:
import re

In [62]:
with open('mails.txt', 'r') as f:
    for line in f.readlines():
        line = line.rstrip()
        convert_corr = re.sub('.*@(?!(edu.hse.ru|hse.ru))', 'Pr1v@cY REstorED@ ', line)
        clear_email = re.split('@', convert_corr)[:-1] # делим по @ и убираем последний (т.е @mail.ru)
        print("@".join(clear_email))

ddrustamova
ejrpgerg
ddrustamova
Pr1v@cY REstorED
Pr1v@cY REstorED
Pr1v@cY REstorED
alena
mila
Pr1v@cY REstorED
Pr1v@cY REstorED


### Погенерируем пароли

Вы наконец-то обеспокоились своей безопасностью в сети и решили обновить все свои пароли. Однако придумывать все самим для всех ваших сайтов вам, конечно, лень. Поэтому вы решили написать программу, которая это сделает за вас!

Сгенерируйте случайный пароль длины N. Убедитесь, что получившийся пароль на самом деле хороший. В нашем случае это:
1. Содержит хотя бы две заглавные буквы подряд
2. Содержит хотя бы две строчные буквы подряд
3. Не содержит пробельные спецсимволы (\t, \n и подобные)
4. Не содержит трех цифр подряд
5. Содержит и русские, и английские буквы 

Ваш код должен генерировать пароли, пока они не удовлетворят заданным условиям.

In [39]:
import sre_yield
import random

In [40]:
symbols = list(sre_yield.AllStrings('([A-Z]|[1-9]|[a-z]|[А-Я]|[а-я]|[\t\n\s\0\r])'))

In [88]:
password = "f3f"
print(re.fullmatch('[0-9]+', password))

None


In [52]:
n = 8
password = "абAB23о2"
while not re.fullmatch(r'[(A-Z|А-Я){2,}(a-z|а-я){2,}[^\d{3}][^\t\n\s\0\r]а-яА-яA-za-z]+', password):
    password = "".join([random.choice(symbols) for i in range(n)])
print(password)

KeyboardInterrupt: 

### Как там героев Федора Михайловича перевели?

1. Вычлените из текста цельные предложения, в которых упоминаются герои по их `Имени Отчеству` (примерно)
2. Отфильтруйте только вопросительные предложения
3. Предложения должны начинаться с `Имени Отчества` (вам поможет positive look-behind)

In [18]:
def get_content() -> tuple:
    """
    return:
        content : str - контент книги
        sentences : list - предложения
    """
    with open('crime-punishment.txt', 'r', encoding='utf-8') as f:
        content = []

        for line in f.readlines()[128:]: # именно с этой строки начинается сама книга
            new_line = line.rstrip()
            if (new_line and 
                not re.match('(PART|CHAPTER) [IV]+', new_line)):
                # я посмотрела файл, там максимум 9 глав\частей
                content.append(new_line)

        content = " ".join(content)
        sentences = re.split(r'\. |! |\? |\.” |\?”|!”|\?’ |!’ |\?” |!” ', content)
    return content, sentences

In [19]:
def get_sent_with_names(sentences: list) -> list:
    """
    return:
        sent_with_names : list - предложения включающие Имя Отчество 
    """
    sent_with_names = []

    for sentence in sentences:
        if re.search('[A-Z]\w+\s[A-Z]\w+', sentence): # имя отчество
            sent_with_names.append(sentence)
    return sent_with_names

In [20]:
def get_start_with_names(sentences) -> list:
    """
    return:
        with_names : list - предложения, начинающиеся с Имени Отчества
    """
    with_names = []
    for sentence in sentences:
        if re.match(r'[A-Z]\w+\s[A-Z]\w+.[\w{\,\;\:}\s]+', sentence):
            with_names.append(sentence)
    return with_names

In [5]:
content, sentences = get_content()
# print(content)

Отфильтровать только вопросительные предложения

In [15]:
only_questions = re.findall(r'[A-Z]+[\w{\,\:\;}\s]*\?', content)
print(only_questions[:5])


['Why am I going there now?', 'Am I capable of _that_?', 'Is _that_ serious?', 'If I am so scared now, what would it be if it somehow came to pass that I were really going to do it?', 'What do you want?']


Предложения включащие Имя Отчество

In [16]:
sent_with_names = get_sent_with_names(only_questions)
print(sent_with_names[:5])

['How much will you give me for the watch, Alyona Ivanovna?', 'Katerina Ivanovna, am I really to do a thing like that?', 'His excellency Ivan Afanasyvitch, do you know him?', 'Oh Lord, why dost Thou receive these men?', 'How are you going to save them from Svidrigaïlovs, from Afanasy Ivanovitch Vahrushin, oh, future millionaire Zeus who would arrange their lives for them?']


Предложения начинающиеся с Имени Отчества

In [21]:
sent_start_names = get_start_with_names(sent_with_names)
sent_start_names[10:15]

['Marfa Petrovna?',
 'What Marfa Petrovna?',
 'Porfiry Petrovitch?',
 'Nikodim Fomitch about me?',
 'Did Porfiry wink at me just now?']

#### Ошибся со временем

Вы написали длинное письмо, где описали свои планы на день с указанием конкретных временных промежутков. Однако позже вы осознали, что на самом деле забыли об одном важном деле, и теперь все даты в вашем письме неверные. Напишите программу, которая заменяет все временные указания на строчку TBD

*Пример:*

`Уважаемый Д.! Если вы к 12:00 вторника не подготовите свой семинар, то уже в 12:00:01 я за себя не отвечаю. 
Нужно нагененировать хотя бы 6 задач и успеть разобрать в идеале 80:100 из них` 

->

`Уважаемый Д.! Если вы к (TBD) вторника не подготовите свой семинар, то уже в (TBD) я за себя не отвечаю. 
Нужно нагененировать хотя бы 6 задач и успеть разобрать в идеале 80:100 из них` 